In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import RobustScaler
from scipy import stats
from tqdm import tqdm

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import datatable as dt
import glob
import os
from tensorflow.python.keras.callbacks import Callback
import keras.backend as K
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
sns.set_style('darkgrid')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
TRAIN_MODEL = False
seed = 1021


tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
df = dt.fread('../input/jane-street-market-prediction/train.csv')
df = df.to_pandas()
df = df.astype({c: np.float32 for c in df.select_dtypes(include='float64').columns}) #limit memory use

In [ ]:
log_top_nan_f = ['feature_84', 'feature_90', 'feature_96','feature_102', 'feature_108', 'feature_114']
nolog_top_nan_f = ['feature_7', 'feature_8', 'feature_17', 'feature_18', 'feature_27', 'feature_28', 'feature_72', 'feature_78']

top_nan_features = log_top_nan_f + nolog_top_nan_f

In [ ]:
Q1_nolog_top_nan_f = df[nolog_top_nan_f].quantile(0.005)
Q3_nolog_top_nan_f = df[nolog_top_nan_f].quantile(0.995)
df[nolog_top_nan_f] = df[nolog_top_nan_f].clip(lower=Q1_nolog_top_nan_f,upper=Q3_nolog_top_nan_f,axis=1)
bias_nolog_top_nan_f = 1 - df[nolog_top_nan_f].min()
df[nolog_top_nan_f] = df[nolog_top_nan_f] + bias_nolog_top_nan_f
df[nolog_top_nan_f].hist(bins=100,figsize=(20,30),layout=(6,3))

In [ ]:
bias_log_top_nan_f = np.exp(1) - df[log_top_nan_f].min()
df[log_top_nan_f] = df[log_top_nan_f] + bias_log_top_nan_f
df[log_top_nan_f] = np.log(df[log_top_nan_f])
df[log_top_nan_f].hist(bins=100,figsize=(20,30),layout=(6,3))

In [ ]:
df[top_nan_features] = df[top_nan_features].fillna(0)
# df[top_nan_features] = df[top_nan_features].fillna(df[top_nan_features].mean())
print(df.shape)
df = df.dropna()
print(df.shape)

In [ ]:
features_to_drop_quantiles = ['feature_46','feature_48','feature_49', 'feature_50','feature_51','feature_54','feature_55','feature_56','feature_57','feature_58','feature_59']
for f in features_to_drop_quantiles:
    df = df.loc[df[f] < df[f].quantile(0.999)].reset_index(drop=True)
print(df.shape)

In [ ]:
features = [c for c in df.columns if 'feature' in c]
resp_cols = ['resp','resp_1','resp_2','resp_3','resp_4']

In [ ]:
val = df.query('400 <= date').reset_index(drop = True)
train = df.query('85 < date < 400').reset_index(drop = True)

rb_scalar = RobustScaler()
rb_scalar.fit(train[features].values)

train = train.query('weight > 0').reset_index(drop = True)
val = val.query('weight > 0').reset_index(drop = True)

X = train[features].values
X_val = val[features].values

X = rb_scalar.transform(X)
X_val = rb_scalar.transform(X_val)

Y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
Y_val = np.stack([(val[c] > 0).astype('int') for c in resp_cols]).T

f_mean = np.mean(train[features].values,axis=0)
X.shape

In [ ]:
def create_autoencoder(input_dim, noise=0.1):
    inp = Input(input_dim)
    encoded = GaussianNoise(noise)(inp)
    
    encoded = Dense(128)(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Activation('relu')(encoded)
#     encoded = Dropout(0.2)(encoded)
    
    encoded = Dense(64)(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Activation('relu')(encoded)
#     encoded = Dropout(0.2)(encoded)
    
    encoded = Dense(32)(encoded)
    encoded = BatchNormalization()(encoded)
    
    decoded = Dense(64,activation='relu')(encoded)
    decoded = BatchNormalization()(decoded)
#     decoded = Dropout(0.2)(decoded)

    decoded = Dense(128,activation='relu')(decoded)
    
    decoded = Dense(input_dim, name='encoder')(decoded)
    
    encoder = Model(inputs=inp,outputs=encoded)
    autoencoder = Model(inputs=inp,outputs=decoded)
    
    autoencoder.compile(optimizer=Adam(0.001),loss='mae')
    return autoencoder,encoder

autoencoder,encoder = create_autoencoder(X.shape[-1])
autoencoder.summary()
autoencoder.load_weights('../input/auto-encoder/autoencoder.hdf5')
encoder.load_weights('../input/auto-encoder/encoder.hdf5')
encoder.trainable = False
autoencoder.trainable = False

In [ ]:
X_pred = autoencoder(X_val)
from matplotlib import pyplot as plt
fig = plt.figure()
plt.plot(X_val[-100:,1])
plt.plot(X_pred[-100:,1])
plt.show()

In [ ]:
# def utility_score(date, weight, resp, action):
#     Pi = np.bincount(date, weight * resp * action)
#     t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
#     u = min(max(t, 0), 6) * np.sum(Pi)
#     return u, np.sum(weight * resp * action)


def utility_score(date, weight, resp, action):
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return u

class UtilityScoreCallback(tf.keras.callbacks.Callback):
    def __init__(self, X, date, weight, resp, stage = 'train'):
        super(Callback, self).__init__()
        self.X = X
        self.date = date
        self.weight = weight
        self.resp = resp
        self.stage = stage

    def on_epoch_end(self, epoch, logs = {}):
        y_preds = self.model(self.X)
        y_preds = np.median(y_preds, axis = 1) # this is for multi-target
#         y_preds = np.squeeze(y_preds)
        action = np.where(y_preds >= 0.5, 1, 0)
        uscore = utility_score(self.date, self.weight, self.resp, action)
        print("Uscores {} : {:.2f} Percent Action 1: {:.2f}".format(self.stage, uscore, np.sum(action)/len(action)))

In [ ]:
# print('scores before training')
# Y_hat = model.predict(X)
# Y_hat = np.median(Y_hat,axis=1)
# # Y_hat = np.squeeze(Y_hat)
# Y_hat[Y_hat>0.5] = 1
# Y_hat[Y_hat<=0.5] = 0

# print(Y_hat.shape)

# print(utility_score(train['date'], train['weight'], train['resp'], Y_hat))

# Y_hat_val = model.predict(X_val)
# Y_hat_val = np.median(Y_hat_val,axis=1)
# # Y_hat_val = np.squeeze(Y_hat_val)
# Y_hat_val[Y_hat_val>0.5] = 1
# Y_hat_val[Y_hat_val<=0.5] = 0

# print(Y_hat_val.shape)
# print(utility_score(val['date'], val['weight'], val['resp'], Y_hat_val))

In [ ]:
def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
#     x = autoencoder(inp)
    x = encoder(inp)
    x = tf.keras.layers.BatchNormalization()(x)
#     x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        optimizer=tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
#         optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate),
#         optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model

batch_size = 5000
hidden_units = [64, 64, 64, 64]
dropout_rates = [0, 0.1, 0.1, 0.1, 0]
label_smoothing = 1e-2
learning_rate = 1e-4

In [ ]:
all_models = []

for i_model in range(5):
    model = create_mlp(X.shape[-1], Y.shape[-1], hidden_units, dropout_rates, label_smoothing, learning_rate)
    model.summary()

    print(X.shape)
    print(Y.shape)
    if TRAIN_MODEL:
        model.fit(X,Y,
                  epochs=500,
                  batch_size=4096,
                  validation_data = (X_val,Y_val),
    #               sample_weight = sample_weight,
                  callbacks = [EarlyStopping('val_loss', patience=50, restore_best_weights=True),
    #                            UtilityScoreCallback(X, train['date'].values, train['weight'].values, train['resp'].values, stage = 'train'),
    #                            UtilityScoreCallback(X_val, val['date'].values, val['weight'].values, val['resp'].values, stage = 'val')
                              ]
    #               callbacks = UtilityScoreCallback(X_val, val['date'].values, val['weight'].values, val['resp'].values, stage = 'val')
                       )
        model.save_weights('./model_' + str(i_model) + '.hdf5')
    else:
        model.load_weights('../input/auto-encoder-deep-model/model_' + str(i_model) + '.hdf5')
    all_models.append(model)

In [ ]:
all_Y_hat_val = []
for model in all_models:
    Y_hat_val = model.predict(X_val)
    Y_hat_val = np.median(Y_hat_val,axis=1)
    Y_hat_val = np.squeeze(Y_hat_val)
    all_Y_hat_val.append(Y_hat_val)
all_Y_hat_val = np.array(all_Y_hat_val)

Y_hat_val = np.mean(all_Y_hat_val,axis=0)

best_util = 0
best_thr = None
for i in range(4500, 5500):
    thr = float(i) / 10000
    Y_hat_val_ = Y_hat_val.copy()
    Y_hat_val_[Y_hat_val_>thr] = 1
    Y_hat_val_[Y_hat_val_<=thr] = 0
    ul = utility_score(val['date'], val['weight'], val['resp'], Y_hat_val_)
#     print(str(thr) + ' ' + str(ul)))
    if ul > best_util:
        best_util = ul
        best_thr = thr
    if i == 5000:
        print(ul)
        print(thr)

print(best_util)
print(best_thr)

In [ ]:
no_log_top_nan_f_ind = [int(f.split('_')[1]) for f in nolog_top_nan_f]
log_top_nan_f_ind = [int(f.split('_')[1]) for f in log_top_nan_f]
top_nan_features_ind = [int(f.split('_')[1]) for f in top_nan_features]
for model in all_models:
    model.call = tf.function(model.call, experimental_relax_shapes=True)
encoder.call = tf.function(encoder.call, experimental_relax_shapes=True)
count = 0


import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_ = test_df.loc[:, features].values
        
        x_ = np.squeeze(x_)
        x_temp = x_[no_log_top_nan_f_ind]
        x_temp = np.clip(x_temp, Q1_nolog_top_nan_f.values, Q3_nolog_top_nan_f.values)
        x_temp = x_temp + bias_nolog_top_nan_f.values
        x_[no_log_top_nan_f_ind] = x_temp
        
        x_[log_top_nan_f_ind] = x_[log_top_nan_f_ind] + bias_log_top_nan_f.values
        x_[log_top_nan_f_ind] = np.log(x_[log_top_nan_f_ind])
        
        x_temp = x_[top_nan_features_ind]
        x_temp[np.isnan(x_temp)] = 0
        x_[top_nan_features_ind] = x_temp
        
        x = x_[np.newaxis, :]


        if np.isnan(x.sum()):
            x = np.nan_to_num(x) + np.isnan(x) * f_mean

        x = rb_scalar.transform(x)
        
        y = np.zeros(len(all_models))
        ii = 0
        for model in all_models:
            y_hat = model(x, training = False).numpy()
            y_hat = np.median(y_hat)
            y[ii] = y_hat
            ii = ii + 1

        y = np.mean(y)
        if y > 0.5:
            pred_df.action = 1
            count = count + 1
        else:
            pred_df.action = 0
    else:
        pred_df.action = 0
    env.predict(pred_df)
print(count)

In [ ]:
# janestreet.competition.make_env.__called__ = False